In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 12 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

09 January 2023 ; 17:54:47


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,17,0,8.8,1.5,3.1,0.500,...,1.21,112.47,111.2,1.27,1.11,112.45,111.24,1.21,1,1
1,Bobby Portis,PF,27,MIL,39,9,26.5,5.9,12.0,0.496,...,1.21,112.47,111.2,1.27,1.11,112.45,111.24,1.21,0,1
3,George Hill,PG,36,MIL,31,0,19.8,1.7,3.8,0.437,...,1.21,112.47,111.2,1.27,1.11,112.45,111.24,1.21,0,1
7,Joe Ingles,SF,35,MIL,9,0,18.6,1.7,5.3,0.313,...,1.21,112.47,111.2,1.27,1.11,112.45,111.24,1.21,0,1
8,Jordan Nwora,SF,24,MIL,29,3,16.0,2.0,5.2,0.375,...,1.21,112.47,111.2,1.27,1.11,112.45,111.24,1.21,0,1


In [5]:
# Set training data
train_data = train_data[train_data['Year'] < 2020] # DELETE THIS LINE
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Christian Wood,28.1,38,13,DAL,21.3,2.373684,6.905263,18.2,4,2.4,3.884211,0.500478
1,Malcolm Brogdon,23.6,34,0,BOS,18.7,2.170588,6.270588,13.5,1,2.7,4.341176,0.264151
2,Alec Burks,22.0,30,0,DET,18.9,2.186667,5.466667,14.0,15,2.8,4.646667,0.203665
3,Bobby Portis,26.5,39,9,MIL,19.0,1.892308,6.938462,14.2,3,2.0,3.574359,0.201222
4,Bones Hyland,20.7,31,1,DEN,15.9,0.793548,2.116129,13.5,1,1.9,1.058065,0.117665


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)

In [9]:
res.head(20)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Christian Wood,28.1,38,13,DAL,21.3,2.373684,6.905263,18.2,4,2.4,3.884211,0.500478
1,Malcolm Brogdon,23.6,34,0,BOS,18.7,2.170588,6.270588,13.5,1,2.7,4.341176,0.264151
2,Alec Burks,22.0,30,0,DET,18.9,2.186667,5.466667,14.0,15,2.8,4.646667,0.203665
3,Bobby Portis,26.5,39,9,MIL,19.0,1.892308,6.938462,14.2,3,2.0,3.574359,0.201222
4,Bones Hyland,20.7,31,1,DEN,15.9,0.793548,2.116129,13.5,1,1.9,1.058065,0.117665
5,Bruce Brown,29.0,38,19,DEN,13.7,1.294737,4.747368,11.1,1,-0.3,2.805263,0.072970
6,Grant Williams,27.2,39,14,BOS,11.9,1.051282,5.887179,8.7,1,-0.7,3.364103,0.070214
7,Tyus Jones,23.6,39,6,MEM,16.3,1.892308,6.097436,10.5,1,1.7,3.994872,0.048208
8,Brandon Clarke,19.5,35,3,MEM,19.4,1.640000,6.325714,10.1,1,0.4,3.748571,0.045456
9,Cole Anthony,26.4,22,3,ORL,14.7,1.118182,4.472727,12.3,13,-0.5,2.236364,0.029731
